# 视频动作分类的中级教程

## 写在前面

在上一篇视频动作分类的初级教程中，我们主要学习了pytorch的基本用法、简单CNN模型的搭建和训练、简单的视频动作分类方法。

在这一篇教程中，我们将更加专注于任务和模型本身，来学习一些更深一层的内容。

### 任务和方法综述

视频任务是计算机视觉任务中非常重要的组成部分。如果从输入和输出角度来看，我们可以不严谨的将其分为三类：
* 定长到定长：比如使用局部信息的视频动作分类，输入确定长度的视频输出确定数量的类别。
* 序列到定长：比如使用全局信息的视频动作分类，输入为视频帧序列输出确定数量的类别。
* 序列到序列：比如视频描述任务，输入视频帧序列，并根据视频内容作出相应的描述。

但是无论任务具体是什么，我们首先都需要对视频进行信息提取，因为视频相邻帧相差很小也就代表视频中有大量的冗余信息。我们所关心的无非只有两条：空间信息和时间信息。空间信息代表了视频主体的类别和位置。时间信息代表了视频主体的变化方式。

对空间信息的抽取几乎可以直接借鉴图像领域的模型和方法，其中最主要的就是CNN，我们提取出视频帧并对每一帧图像进行处理，就能够很好的获得视频的空间信息。但是单独处理并没有考虑到帧与帧之间的时间信息，并不能很好的表达视频内容。

对时间信息的抽取要更关注帧与帧之间的变化。其中最重要的方法就是光流法，通过光流估计算出相邻帧之间的运动关系从而表现动作的概念。所以融合RGB流和光流的双流法就成为视频任务中重要的一个分支。而另外一个重要分支就是3D巻积，直接把多个帧融合到一起做巻积操作也是一种解决方法，那么如何堆叠帧就是首先需要解决的问题。

对于时序信息的处理，往往RNN要比CNN更占优势。所以在2d做CNN抽取在1d时间维度融合RNN也是一个视频处理的重要方法之一。大概在15年就提出并应用了LSTM融合CNN的视频处理框架，并且在近年来不断的完善和改进。

近年来Attention发展非常迅速，并且在很多任务中都能够代替RNN的角色。所以已经有很多模型尝试在视频任务中引入Attention。尤其是类似BERT的大型预训练网络也逐渐部署到视频任务中，比如videoBERT。还有一些知识图谱相关的模型和方法也在逐渐引入。所以视频任务的模型和方法依旧快速发展。

本次教程将主要使用3D巻积方法来完成对视频的动作分类任务。

### 数据集综述


当前常用的视频数据集数量非常多，可以先通过数据规模来分类：
* 较小的比如：
    * [HMDB51](http://serre-lab.clps.brown.edu/wp-content/uploads/2012/08/Kuehne_etal_iccv11.pdf): 51个类别和6,766个视频片段。
    * [UCF101](https://www.crcv.ucf.edu/papers/UCF101_CRCV-TR-12-01.pdf)：101个动作类别，13,320个视频片段。
* 更大的比如
    * [Sports-1M](https://cs.stanford.edu/people/karpathy/deepvideo/deepvideo_cvpr2014.pdf): 487种动作类别，1,100,000个视频片段。
    * [FCVID](https://arxiv.org/abs/1502.07209)：239种动作类别，91,223个视频片段。
* 还有特别大的：
    * [ActivityNet](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Heilbron_ActivityNet_A_Large-Scale_2015_CVPR_paper.pdf)：200个动作类别，648小时。
    * [Youtube-8M](https://arxiv.org/abs/1609.08675)：4716 种动作类别，8,000,000个视频片段。
    * [Kinectics](https://arxiv.org/abs/1705.06950)：600种动作类别，500,000个视频片段。

以上列出的仅为视频领域非常经典的数据。近年来随着方法的改进和硬件的升级数据集的规模变得越来越大。而在对视频初步处理之后数据所占的空间会更加巨大，以UCF101数据举例，解压后的原数据将近10G，对RGB数据进行提取、变换等数据操作后原数据就将近20G，再对视频进行光流提取，光流信息会大于20G。所以最终要应用到训练中的数据很可能在50G左右，这个数据量对于硬件的要求还是非常高的。

视频数据还分为剪辑视频和未剪辑视频。经过剪辑的视频往往时长较短，包含主体明确且唯一，目标动作占据非常主要的部分，并且视频时长和动作时长基本一致。这样的视频数据分类较为容易，例如UCF101的全部数据都是经过剪辑的。而未剪辑的视频数据中动作出现的时间和位置都不确定，具有更大的分类难度。

在近年来模型和方法的不断发展中，视频任务和数据粒度也在从粗转到细。比如早期的分类任务仅仅需要分出一个类别，并且类和类之间关联性很小。这也就导致了很多数据和方法并不是对动作本身分类，而是对特定物体、特定场景、特定光线甚至特定角度等进行分类。而随着任务和数据的不断细化，分类粒度越来越小。比如要对具体动作的不同阶段进行分类。这就要求模型具有真正的区分主体和背景、动作的各个部分的能力。

受限与硬件环境，为了能够专注于模型和方法本身，本次教程我们依旧使用的数据集是精简版的UCF101数据集，可以在[这里](https://www.kesci.com/home/dataset/5f21c369d278b1002c21a8d8)下载，如果你能够科学上网也可以在[这里](https://drive.google.com/file/d/1KmbiMoDFNXc2xxfm--F_Qz2rpn4lkaZx/view?usp=sharing)。该数据在UCF101数据的基础上选择了十个动作类别的数据，在每个视频数据中提取了三帧内容并将其压缩至低分辨率。从而将整个数据规模控制到一个较小的范围，便于在各种设备上进行学习和实验。

下载后解压到```data```目录下。可以看到数据大约按照6:2:2的比例划分训练集、验证集、测试集，训练集、验证集、训练集的数据条数如下表所示。

数据集 | 训练集 | 验证集 | 测试集 | 总数
:-: | :-: | :-: | :-:  | :-:
数据量 | 7770 | 2230 | 3270 | 13270

#### 运行环境

我们在Python3.7环境下测试了本教程代码。需要的第三方模块和版本包括：
```
pytorch
torchvision
PIL
numpy
scipy
```
可以使用pip或conda工具安装上述模块。推荐尝试使用GPU来加速代码的训练，这需要使用nvidia显卡并且搭建相应的cuda环境，当然只使用cpu也能完成本次的学习。

如果可以使用科学上网，我们将非常推荐使用Google的Colab平台。该平台提供了免费的服务器和GPU资源，可以直接运行本教程。

#### 总体概述

本教程包括以下内容：从原始的数据文件中加载数据、对数据进行预处理、神经网络的搭建、训练分类器、结果展示。

#### 建议

建议学习时长：45分钟

建议对代码部分反复修改和尝试。

## 模型概述

我们首先将延续在初级教程中使用的简单CNN模型，我们尝试改进该模型使其具有更好的表达能力。

然后，我们将使用3D巻积代替之前的2D巻积来更好的抽取时序信息。

## 接下来我们就开始吧！
首先，我们导入实验所需要的第三方库：

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader,sampler,Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
import timeit
from PIL import Image
import os
import numpy as np
import scipy.io
import torchvision.models.inception as inception

## 数据加载

在这一部分，我们先使用和初级教程中相同的Dataloader。

首先，加载数据中的整体配置文件，并且打印出训练集和验证集的大小。


In [ ]:
label_mat=scipy.io.loadmat('./data/q3_2_data.mat')
label_train=label_mat['trLb']
print('train len：',len(label_train))
label_val=label_mat['valLb']
print('val len: ',len(label_val))

train len： 7770
val len:  2230


### Dataset类

然后完成Dataset子类的编写。

In [ ]:
class ActionDataset(Dataset):
    """Action dataset."""

    def __init__(self,  root_dir, labels=[], transform=None):
        """
        Args:
            root_dir (string): 整个数据的路径。
            labels(list): 图片的标签。
            transform (callable, optional): 想要对数据进行的处理函数。
        """
        self.root_dir = root_dir
        self.transform = transform
        self.length=len(os.listdir(self.root_dir))
        self.labels=labels

    def __len__(self):      # 该方法只需返回数据的数量。
        return self.length*3    # 因为每个视频片段都包含3帧。

    def __getitem__(self, idx):     # 该方法需要返回一个数据。
        
        folder=idx//3+1
        imidx=idx%3+1
        folder=format(folder,'05d')
        imgname=str(imidx)+'.jpg'
        img_path = os.path.join(self.root_dir,folder,imgname)
        image = Image.open(img_path)

        if len(self.labels)!=0:
            Label=self.labels[idx//3][0]-1
        if self.transform:      # 如果要先对数据进行预处理，则经过transform函数。
            image = self.transform(image)
        if len(self.labels)!=0:
            sample={'image':image,'img_path':img_path,'Label':Label}
        else:
            sample={'image':image,'img_path':img_path}
        return sample



### Dataloader类
对子类简单的修改就完成了对数据的封装。现在我们可以通过一个循环测试一下：

In [ ]:
image_dataset=ActionDataset(root_dir='./data/trainClips/', labels=label_train,transform=T.ToTensor())
# torchvision.transforms中定义了非常多对图像的预处理方法，这里使用的ToTensor方法为将0～255的RGB值映射到0～1的Tensor类型。
for i in range(3):
    sample=image_dataset[i]
    print(sample['image'].shape)
    print(sample['Label'])
    print(sample['img_path'])

torch.Size([3, 64, 64])
0.0
./data/trainClips/00001/1.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00001/2.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00001/3.jpg


最后封装Dataloader类并测试。

In [ ]:
image_dataloader = DataLoader(image_dataset, batch_size=4,
                        shuffle=True, num_workers=4)

for i,sample in enumerate(image_dataloader):
    sample['image']=sample['image']
    print(i,sample['image'].shape,sample['img_path'],sample['Label'])
    if i>5: 
        break

0 torch.Size([4, 3, 64, 64]) ['./data/trainClips/03091/1.jpg', './data/trainClips/02197/1.jpg', './data/trainClips/05940/2.jpg', './data/trainClips/03296/3.jpg'] tensor([3., 2., 7., 3.], dtype=torch.float64)
1 torch.Size([4, 3, 64, 64]) ['./data/trainClips/02173/1.jpg', './data/trainClips/00095/2.jpg', './data/trainClips/01739/1.jpg', './data/trainClips/05789/3.jpg'] tensor([2., 0., 1., 7.], dtype=torch.float64)
2 torch.Size([4, 3, 64, 64]) ['./data/trainClips/04754/3.jpg', './data/trainClips/03760/3.jpg', './data/trainClips/03440/2.jpg', './data/trainClips/02985/1.jpg'] tensor([5., 4., 3., 3.], dtype=torch.float64)
3 torch.Size([4, 3, 64, 64]) ['./data/trainClips/02569/2.jpg', './data/trainClips/03850/3.jpg', './data/trainClips/01659/3.jpg', './data/trainClips/02833/2.jpg'] tensor([2., 4., 1., 3.], dtype=torch.float64)
4 torch.Size([4, 3, 64, 64]) ['./data/trainClips/06912/2.jpg', './data/trainClips/04246/2.jpg', './data/trainClips/00848/2.jpg', './data/trainClips/07680/1.jpg'] tensor

我们发现每一次调用都能够自动返回batch size的数据，并且数据被完全打乱。同时还实现了多线程加载，加载速度提升了很多。

所以我们使用将训练集、验证集和测试集全部封装起来：

In [ ]:
image_dataset_train=ActionDataset(root_dir='./data/trainClips/',labels=label_train,transform=T.ToTensor())

image_dataloader_train = DataLoader(image_dataset_train, batch_size=32,
                        shuffle=True, num_workers=4)
image_dataset_val=ActionDataset(root_dir='./data/valClips/',labels=label_val,transform=T.ToTensor())

image_dataloader_val = DataLoader(image_dataset_val, batch_size=32,
                        shuffle=False, num_workers=4)
image_dataset_test=ActionDataset(root_dir='./data/testClips/',labels=[],transform=T.ToTensor())

image_dataloader_test = DataLoader(image_dataset_test, batch_size=32,
                        shuffle=False, num_workers=4)

In [ ]:
dtype = torch.FloatTensor # 这是pytorch所支持的cpu数据类型中的浮点数类型。

print_every = 100   # 这个参数用于控制loss的打印频率，因为我们需要在训练过程中不断的对loss进行检测。

def reset(m):   # 这是模型参数的初始化
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

## 数据解释和处理

在之前的步骤中我们也看到了最终要使用的数据tensor的形状，其中包含四个维度：
* N：batch size大小。
* C：每帧图像的通道数。
* H：每帧图像的高。
* W：每帧图像的宽。

这就是神经网络的输入形状。我们注意到输入过程中的每一个像素点和原图像都是一一对应的，唯一不同的就是我们将原来较大的RGB值缩放到一个小于1的浮点数，这是模型所需要的。

而在经过多个巻积层的提取之后，我们需要使用全连接层来完成最终的分类判断。此时的tensor的各个维度已经变得和原来截然不同，并且已经没有了一一对应的特点。换句话说，最终的输出的各个维度究竟代表了什么信息只有神经网络自己知道。所以tensor的形状也就不那么重要了。

所以为了便于全连接层的连接，我们定义一个Flatten函数，来将除了N以外的那些我们不知道代表什么的数据统统“展开”到一起。
 

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # 读取各个维度。 
        return x.view(N, -1)  # -1代表除了特殊声明过的以外的全部维度。

## 改进第一个模型！

还记得我们第一篇教程中设置的那个模型吗：

* 7x7 巻积层，8 filters，步长为1。
* ReLU 激活层。
* 2x2 最大池化层，步长为2。
* 7x7 巻积层，16 filters，步长为1。
* ReLU 激活层。
* 2x2 最大池化层，步长为2。
* Flatten 层。
* 2x2 最大池化层，步长为2。
* 全连接层。

整体的结构是非常经典的：巻积+激活+池化结构的堆叠，逐步提取特征值。

他的设置代码如下：


In [ ]:
fixed_model_base = nn.Sequential( 
                nn.Conv2d(3, 8, kernel_size=7, stride=1), #3*64*64 -> 8*58*58
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, stride = 2),    # 8*58*58 -> 8*29*29
                nn.Conv2d(8, 16, kernel_size=7, stride=1), # 8*29*29 -> 16*23*23
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, stride = 2), # 16*23*23 -> 16*11*11
                Flatten(),
                nn.ReLU(inplace=True),
                nn.Linear(1936, 10)     # 1936 = 16*11*11
            )
# 这里模型base.type()方法是设定模型使用的数据类型，之前设定的cpu的Float类型。
# 如果想要在GPU上训练则需要设定cuda版本的Float类型。
fixed_model = fixed_model_base.type(dtype)

### 训练和验证模块的编写

同样和第一篇教程中相似，我们编写出能够自动训练和评估效果的模块：

In [ ]:
def train(model, loss_fn, optimizer, dataloader, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))

        # 在验证集上验证模型效果
        check_accuracy(fixed_model, image_dataloader_val)
        
        model.train() # 模型的.train()方法让模型进入训练模式，参数保留梯度，dropout层等部分正常工作。
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['image'])   # 取得一个batch的图像数据。
            y_var = Variable(sample['Label'].long()) # 取得对应的标签。

            scores = model(x_var)   # 得到输出。
            
            loss = loss_fn(scores, y_var)   # 计算loss。
            if (t + 1) % print_every == 0:  # 每隔一段时间打印一次loss。
                print('t = %d, loss = %.4f' % (t + 1, loss.item()))

            # 三步更新参数。
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy(model, loader):

    num_correct = 0
    num_samples = 0

    model.eval() # 模型的.eval()方法切换进入评测模式，对应的dropout等部分将停止工作。
    for t, sample in enumerate(loader):
        x_var = Variable(sample['image'])
        y_var = sample['Label']
       
        scores = model(x_var)
        _, preds = scores.data.max(1) # 找到可能最高的标签作为输出。

        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))


接下来设定本次实验选用的损失函数和优化器。


In [ ]:
optimizer = torch.optim.RMSprop(fixed_model_base.parameters(), lr = 0.0001)

loss_fn = nn.CrossEntropyLoss()

### 训练并验证

接下来是同样的训练和验证步骤，如果你认真做过第一次实验，你将会非常熟悉这个模型的各个细节。

如果你还不是很熟悉各个超参数是如何设置的，以及模型的训练和优化步骤的话，我希望你能够停下来，好好的调整各个参数并且反复尝试，这对你的训练经验和模型理解是非常有帮助的。


In [ ]:
torch.random.manual_seed(54321)
fixed_model.cpu()
fixed_model.apply(reset) 
fixed_model.train() 
train(fixed_model, loss_fn, optimizer,image_dataloader_train, num_epochs=5) 

## 模型改进

如果你多次尝试了上面的训练过程就会发现，一些好的参数的组合能够给模型带来一定的提升，但是这些提升看起来都很有限。这是因为我们的主体网络的结构还比较简单，不能很好的表达抽象特征。这就需要我们对模型本身做一些调整。

下面让我们来尝试一个更加复杂的模型吧！

这次我们的模型层数会更深，参数量更大。而过大的参数并不是一个好事，因为我们要防止模型在训练集上过拟合的情况发生，所以我们相应的加入一些BatchNorm和Dropout层，来稳定我们的训练过程。

这里的模型只是给出的一个深层模型的框架，主要为了介绍BatchNorm和Dropout层，模型的设计也并不是最合理的。我们非常希望你能够自己动手去改变模型的结构，自己带入公式计算各层的尺寸并且通过测试是一项非常重要的能力。

同时我们很希望你能够自己去尝试：
* 深层与浅层的区别。
* 大巻积核和小巻积核的区别。
* 巻积、激活、池化、norm各层先后顺序的差异。
* 全连接层数的区别。

相信如果亲自对上述问题进行尝试你将会有一些收获。

In [ ]:
fixed_model_base = nn.Sequential( 
                nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, stride = 2),
                nn.BatchNorm2d(64),
                nn.Dropout2d(0.3),
                nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, stride = 2),
                nn.BatchNorm2d(64),
                nn.Dropout2d(0.2),
                nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, stride=2),
                nn.BatchNorm2d(32),
                Flatten(),
                nn.Linear(2048, 512),
                nn.Linear(512, 64),
                nn.Linear(64, 10),
                nn.LogSoftmax()
            )


fixed_model = fixed_model_base.type(dtype)

optimizer = torch.optim.RMSprop(fixed_model_base.parameters(), lr = 0.0001)

loss_fn = nn.CrossEntropyLoss()

同样的先测试模型结构的正确性：

In [ ]:
x = torch.randn(32, 3, 64, 64).type(dtype)
x_var = Variable(x.type(dtype)) 
ans = fixed_model(x_var)         

print(np.array(ans.size()))
np.array_equal(np.array(ans.size()), np.array([32, 10]))   

[32 10]


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


True

训练并测试：

In [ ]:
torch.random.manual_seed(54321)
fixed_model.cpu()
fixed_model.apply(reset) 
fixed_model.train() 
train(fixed_model, loss_fn, optimizer,image_dataloader_train, num_epochs=12) 
check_accuracy(fixed_model, image_dataloader_test)# check accuracy on the training set


## 使用3D巻积来升级模型！

正如前面所提到的，2d巻积并不能很好的抽取时序信息，所以我们尝试使用3d巻积。

在本次教程中，我们主要使用3帧堆叠出输入的数据块，在实际应用中可以堆叠更多的帧数来覆盖更大的时序信息范围。

首先让我们来更新我们的数据加载工具：

In [ ]:
class ActionClipDataset(Dataset):

    def __init__(self, root_dir, labels=[], transform=None):
        """
        Args:
            root_dir (string): 整个数据的路径。
            labels(list): 图片的标签。
            transform (callable, optional): 想要对数据进行的处理函数。
        """
        
        self.root_dir = root_dir
        self.transform = transform
        self.length=len(os.listdir(self.root_dir))
        self.labels=labels

    def __len__(self):  # 同样的重载__len__方法。
        return self.length  # 此时的长度就不再是之前的三倍了，对每一个clip我们都要用到三章图像。

    def __getitem__(self, idx):     # 同样的重载__getitem__方法。
        
        folder=idx+1
        folder=format(folder,'05d')
        clip=[]
        if len(self.labels)!=0:
            Label=self.labels[idx][0]-1
        for i in range(3):      # 循环提取三张图像。
            imidx=i+1
            imgname=str(imidx)+'.jpg'
            img_path = os.path.join(self.root_dir,
                                    folder,imgname)
            image = Image.open(img_path)
            image=np.array(image)
            clip.append(image)
        if self.transform:
            clip=np.asarray(clip)
            clip=np.transpose(clip, (0,3,1,2))
            clip = torch.from_numpy(np.asarray(clip))
        if len(self.labels)!=0:
            sample={'clip':clip,'Label':Label,'folder':folder}
        else:
            sample={'clip':clip,'folder':folder}
        return sample

clip_dataset=ActionClipDataset(root_dir='./data/trainClips/',\
                               labels=label_train,transform=T.ToTensor())
for i in range(3):
    sample=clip_dataset[i]
    print(sample['clip'].shape)
    print(sample['Label'])
    print(sample['folder'])

经过验证我们发现每个数据点都包含了三张图像。

接着我们来封装Dataloader类：

In [ ]:
clip_dataloader = DataLoader(clip_dataset, batch_size=4,
                        shuffle=True, num_workers=4)


for i,sample in enumerate(clip_dataloader):
    print(i,sample['clip'].shape,sample['folder'],sample['Label'])
    if i>5: 
        break

然后实例化三个数据集：

In [ ]:
clip_dataset_train=ActionClipDataset(root_dir='./data/trainClips/',labels=label_train,transform=T.ToTensor())

clip_dataloader_train = DataLoader(clip_dataset_train, batch_size=16,
                        shuffle=True, num_workers=4)
clip_dataset_val=ActionClipDataset(root_dir='./data/valClips/',labels=label_val,transform=T.ToTensor())

clip_dataloader_val = DataLoader(clip_dataset_val, batch_size=16,
                        shuffle=True, num_workers=4)
clip_dataset_test=ActionClipDataset(root_dir='./data/testClips/',labels=[],transform=T.ToTensor())

clip_dataloader_test = DataLoader(clip_dataset_test, batch_size=16,
                        shuffle=False, num_workers=4)

现在的Flatten函数和之前要有些许不同：

In [ ]:
class Flatten3d(nn.Module):
    def forward(self, x):
        N, C, D, H, W = x.size() 
        return x.view(N, -1)  

接下来搭建一个简单的3d巻积神经网络。

在pytorch中直接使用Conv3d就能够快速添加3d巻积层。

这次在全连接层之后还添加了softmax层增强分类能力，你可以去掉它看看模型效果会变好还是变坏。

这次的神经网络结构依旧只是一个能够运行的结构，它还有非常大的改进空间。如果你有认真去尝试过之前的几次模型调整，那么你现在应该会有很多改进的想法，动手尝试一下可不可行吧！

In [ ]:
fixed_model_3d = nn.Sequential( 
    nn.Conv3d(in_channels = 3, out_channels = 50, kernel_size = 2, stride = 1),
    nn.ReLU(inplace=True),
    nn.MaxPool3d((1, 2, 2), stride = 2),
    nn.Conv3d(in_channels = 50, out_channels = 100, kernel_size = (1, 3, 3), stride = 1),
    nn.ReLU(inplace = True),
    nn.MaxPool3d((1, 3, 3), stride = 2),
    nn.Dropout3d(0.1),
    Flatten3d(),
    nn.ReLU(inplace=True),
    nn.Linear(19600, 10),
    nn.LogSoftmax() 
)

fixed_model_3d = fixed_model_3d.type(dtype)
x = torch.randn(32,3, 3, 64, 64).type(dtype)
x_var = Variable(x).type(dtype)
ans = fixed_model_3d(x_var) 
np.array_equal(np.array(ans.size()), np.array([32, 10]))

老样子的训练和测试模块。

建议你先使用与2d巻积模型相同的loss函数和优化器，看看更换了3d巻积之后模型的效果到底有哪些变化。然后再尝试去更换不同的loss函数和优化器种类，看看3d巻积和2d巻积模型最适合的组合是不是一样呢。

In [ ]:
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.RMSprop(fixed_model_3d.parameters(), lr=1e-4)

In [ ]:
def train_3d(model, loss_fn, optimizer,dataloader,num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        check_accuracy_3d(fixed_model_3d, clip_dataloader_val)
        model.train()
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['clip'].type(dtype))
            y_var = Variable(sample['Label'].type(dtype).long())

            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.item()))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy_3d(model, loader):

    num_correct = 0
    num_samples = 0
    model.eval() 
    for t, sample in enumerate(loader):
        x_var = Variable(sample['clip'].type(dtype))
        y_var = sample['Label'].type(dtype)
        y_var=y_var.cpu()
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
 
        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
torch.cuda.random.manual_seed(782374)
fixed_model_3d.apply(reset) 
fixed_model_3d.train() 
train_3d(fixed_model_3d, loss_fn, optimizer,clip_dataloader_train, num_epochs=5) 
fixed_model_3d.eval() 
check_accuracy_3d(fixed_model_3d, clip_dataloader_val)

#### 总结
本次教程的任务在上次的基础之上有了提升和改进。在了解了pytorch基本用法和神经网络模型的搭建和训练的基本步骤之后，我们希望你能够更关注任务本身和模型本身。

经过本次教程的学习，我们希望你能对CNN模型和视频任务有更进一步的认识和了解。希望你在大量的尝试之后能够发现模型不同结构、参数、方法带来的影响。也希望你能够经一部了解视频和图像的差距，时序信息在视频信息中的重要性。

更多的，如果你认真的对网络参数和网络结构进行了多次调整和测试，我们希望你能够继续探索更深的问题：
* 巻积核大小的影响。
* 池化层和单核巻积。
* Normalization的多种方法。
* 分类器能不能做的更好，比如在后面添加softmax、SVM或者CRF等。
* 正则化。
* 不同优化器的选择和参数设置。
* 不同激活函数的效果。
* 数据增强方法。
* 层数越深参数越多越好吗。
* 当前网络距离那些更成熟的网络还有那些差距，如ResNet、VGG等。
* 3d巻积和2d巻积的异同。
* 如何能够更有效的提取时序信息，降低信息的冗余度。

非常希望你能够认真的思考和探索这些问题，亲自动手尝试将是学习过程中最最重要的事情。